In [1]:
import sounddevice as sd
import numpy as np
import librosa
from tensorflow.keras.models import load_model

# Modelin yüklenmesi
model = load_model('modelv3.h5')  # Eğitilmiş modelin dosya yolu
def extract_features(sound_signal, sample_rate):
    mfcc_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=13)
    pad_width = 44 - mfcc_features.shape[1]
    if pad_width > 0:
        mfccs_scaled_features = np.pad(mfcc_features, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs_scaled_features = mfcc_features[:, :44]  # Gerekiyorsa MFCC'lerin boyutunu kırp
    return mfccs_scaled_features.reshape(1, 44, 13)

# Mikrofondan ses verisi almak için gerekli parametreler
RATE = 44100
DURATION = 5  # Kayıt süresi (saniye)

# Ses yakalama işlemi
print("Lütfen konuşun...")
sound_signal = sd.rec(int(DURATION * RATE), samplerate=RATE, channels=1, dtype='float32')
sd.wait()  # Ses yakalamanın tamamlanmasını bekleyin
print("Kayıt tamamlandı.")

# Ses verisinin özelliklere dönüştürülmesi
mfccs_scaled_features = extract_features(np.squeeze(sound_signal), RATE)

# Modelin kullanılması ve sonucun alınması
result_array = model.predict(mfccs_scaled_features)
result_classes = ['tahakocer','mehmetkocoglu']

# Tahmin edilen sanatçıları ve olasılıklarını gösterme
predicted_artist_indices = np.argsort(result_array[0])[::-1]  # Olasılıklara göre sıralanmış tahmin edilen sınıf indisleri

print("Tahmin edilen sanatçılar ve olasılıkları:")
for index in predicted_artist_indices:
    predicted_artist = result_classes[index]
    probability = result_array[0][index]
    print(f"{predicted_artist}: {probability:.2f}")


Lütfen konuşun...
Kayıt tamamlandı.


NameError: name 'recognizer' is not defined